## Assignment 1
### Name: Sake venkata vignan kumar
### Roll Number:20CS30070

In [35]:
# import all the necessary libraries here
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [36]:
df = pd.read_csv('../../dataset/cross-validation.csv')
print(df.shape) 

(614, 13)


In [37]:
df = df.sample(frac=1, random_state=56)

# Identify the columns that need to be converted to float
numeric_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

# Calculate the means for numeric columns
numeric_means = df[numeric_columns].mean()

# Replace missing values in numeric columns with the calculated means
df[numeric_columns] = df[numeric_columns].fillna(numeric_means)

# Encoding map
Loan_encoding_map = {'Y': 0, 'N': 1}
Gender_encoding_map = {'Male':0, 'Female':1}
Married_encoding_map = {'Yes':1, 'No':0}
Education_encoding_map = {'Graduate':1, 'Not Graduate':0}
SelfEmp_encoding_map = {'Yes':1, 'No':0}
Property_encoding_map = {'Urban':1, 'Semi Urban':0, 'Rural':2}
dep_encoding_map = {'3+':4}

# Map the column
df['Loan_Status'] = df['Loan_Status'].map(Loan_encoding_map)
df['Gender'] = df['Gender'].map(Gender_encoding_map)
df['Married'] = df['Married'].map(Married_encoding_map)
df['Education'] = df['Education'].map(Education_encoding_map)
df['Self_Employed'] = df['Self_Employed'].map(SelfEmp_encoding_map)
df['Property_Area'] = df['Property_Area'].map(Property_encoding_map)
df['Dependents'] = df['Dependents'].map(dep_encoding_map)

X = df.drop(columns=['Loan_Status','Loan_ID'])
y = df['Loan_Status']

# Convert DataFrame to NumPy arrays
X = df.drop(columns=['Loan_Status','Loan_ID']).values
y = df['Loan_Status'].values

# Split dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert only the numeric columns to float
X_train[:, :len(numeric_columns)] = X_train[:, :len(numeric_columns)].astype(float)
X_test[:, :len(numeric_columns)] = X_test[:, :len(numeric_columns)].astype(float)

# Logistic sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Logistic regression training function
def train_logistic_regression(X, y, learning_rate, num_iterations):
    num_samples, num_features = X.shape
    weights = np.zeros(num_features)
    
    for _ in range(num_iterations):
        z = np.dot(X, weights)
        predictions = sigmoid(z)
        error = predictions - y
        gradient = np.dot(X.T, error) / num_samples
        weights -= learning_rate * gradient
    
    return weights

# Cross-validation logic
num_folds = 5
fold_size = len(X_train) // num_folds
accuracy_scores = []
precision_scores = []
recall_scores = []

for i in range(num_folds):
    fold_start = i * fold_size
    fold_end = fold_start + fold_size
    fold_X_val = X_train[fold_start:fold_end]
    fold_y_val = y_train[fold_start:fold_end]
    fold_X_train = np.concatenate((X_train[:fold_start], X_train[fold_end:]))
    fold_y_train = np.concatenate((y_train[:fold_start], y_train[fold_end:]))

    # Convert fold_X_train and fold_y_train to numpy arrays
    fold_X_train = np.array(fold_X_train)
    fold_y_train = np.array(fold_y_train)

    # Train the logistic regression model on the current fold
    learning_rate = 0.001
    num_iterations = 1000
    weights = train_logistic_regression(fold_X_train, fold_y_train, learning_rate, num_iterations)

    # Predict class labels for the validation fold
    fold_predictions = (np.dot(fold_X_val, weights) >= 0.5).astype(int)

    # Calculate accuracy, precision, and recall for class 1 (good)
    true_positive = np.sum((fold_predictions == 1) & (fold_y_val == 1))
    false_positive = np.sum((fold_predictions == 1) & (fold_y_val == 0))
    false_negative = np.sum((fold_predictions == 0) & (fold_y_val == 1))

    accuracy = (true_positive + np.sum(fold_predictions == 0)) / len(fold_y_val)

    # Check for division by zero
    if true_positive + false_positive == 0:
        precision = 0.0
    else:
        precision = true_positive / (true_positive + false_positive)

    recall = true_positive / (true_positive + false_negative)

    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)

# Calculate mean accuracy, precision, and recall across all folds
mean_accuracy = np.mean(accuracy_scores)
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)

print(f"Mean Accuracy: {mean_accuracy:.4f}")
print(f"Mean Precision: {mean_precision:.4f}")
print(f"Mean Recall: {mean_recall:.4f}")


Mean Accuracy: 1.0000
Mean Precision: 0.0000
Mean Recall: 0.0000
